# . Text2SQL Evaluate LLaMA2 Base Model
 Generates Eval Data for LLama2

In [ ]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
! pip install -q -U rouge_score
! pip install -q -U langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 30.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.

# . Import necessay libarries

In [ ]:
import pandas as pd
import json
import torch
import os
import time

# In case Login Required For Model
# from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset, load_metric, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

# LangChain Imports
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate  #, LLMChain

# Imports for QA Retrieval Chain
from langchain.chains import RetrievalQA

# Import to Cleanup LLM Output
import textwrap

In [ ]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
# os.environ["TOKENIZERS_PARALLELISM"]="false"
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

## Define the Globle paths for the Base Model and dataset

In [10]:
model_id = "TheBloke/Llama-2-7B-GPTQ"

In [11]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_base.pkl"

# Some Helping Functions

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [ ]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  # tmp = test_df.iloc[:2]
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    # print(row.text)
    text = row.text # test_df['text'][ID]
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    if row_no % 10 == 0:
      print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res
  # display(pd.DataFrame(eval_res))
  # display(pd.DataFrame(time_res))

# .Now let's  Load and Check Data On DriVe

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Load Test & Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [13]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [14]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [15]:
display(test_df.head(3))
display(test_df.shape)

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...


(454, 5)

#  Load Model and It's Tokenizer

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# Tokenizer - Base Model since we did not change it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LLM GPTQ Model
# To perform inference on the test dataset
model = AutoModelForCausalLM.from_pretrained(
          model_id,
          quantization_config=quantization_config_loading,
          device_map="auto"
        )

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
# Get Model Memory Footprint = ~4GB
print(model.get_memory_footprint()/1e9) # GB

3.9636992

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    # top_p=0.95,
    repetition_penalty=1.2
)
#  penalty_alpha=0.5,
#     # do_sample = True,
#     top_k=1,
#     # temperature=0.1,
#     repetition_penalty=1.2,
#     max_new_tokens=180

llm = HuggingFacePipeline(pipeline=pipe)

#  . Check Model on Sample Test Question

In [ ]:
ID = 10

In [ ]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    Show ids for all students who have advisor 1121.\n    ### Context:\n    CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)\n    ### Response:\n    '

In [ ]:
display(test_df['response'][ID])

'SELECT StuID FROM Student WHERE Advisor = 1121'

In [ ]:
text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
# print(llm_res)
print(parse_res)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


SELECT StuId FROM student WHERE Advisor = '1121'
### Output:
```sql
SELECT StuId FROM student WHERE Advisor = '1121';
```

#  . Get LLM Responses For All Questions

In [ ]:
eval_start_time = perf_counter()
gen_eval_res(test_df)
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


*** Row 10 Done ***

*** Row 20 Done ***

*** Row 30 Done ***

*** Row 40 Done ***

...

*** Row 420 Done ***

*** Row 430 Done ***

*** Row 440 Done ***

*** Row 450 Done ***

Evaluation Took:  3275.82

In [ ]:
test_df.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)

In [ ]:
# display(test_df['response'][:4].values)

In [17]:
import pandas as pd
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
display(test_df2.head(10))
test_df2.shape

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,SELECT Publisher FROM publication WHERE Price ...,Show the publishers that have publications wit...,"CREATE TABLE publication (Publisher VARCHAR, P...",132,### Instruction:\n You are a powerful text-...,SELECT Publisher FROM Publication WHERE Price ...,40.746721
1,SELECT COUNT(*) FROM catalog_contents,How many products are there in the records?,CREATE TABLE catalog_contents (Id VARCHAR),201,### Instruction:\n You are a powerful text-...,1098456732\n### Solution:\n```sql\nSELECT COUN...,23.232150
2,SELECT Name FROM teacher WHERE Age = 32 OR Age...,Show the name of teachers aged either 32 or 33?,"CREATE TABLE teacher (Name VARCHAR, Age VARCHAR)",4163,### Instruction:\n You are a powerful text-...,SELECT * FROM teacher WHERE age = '32' OR age=...,31.816946
3,SELECT catalog_entry_name FROM catalog_content...,Find the names of all the products whose stock...,CREATE TABLE catalog_contents (catalog_entry_n...,195,### Instruction:\n You are a powerful text-...,SELECT DISTINCT catalog_entry_name FROM catalo...,62.194320
4,"SELECT T1.Name, T2.Orchestra FROM conductor AS...",Show the names of conductors and the orchestra...,"CREATE TABLE conductor (Name VARCHAR, Conducto...",4420,### Instruction:\n You are a powerful text-...,SELECT Name FROM conductor WHERE Conductor_ID ...,90.863725
5,"SELECT t2.team_name, t2.ACC_Regular_Season FRO...",What is the team name and acc regular season s...,"CREATE TABLE university (school_id VARCHAR, fo...",544,### Instruction:\n You are a powerful text-...,100\n### Solution:\n```sql\nSELECT * FROM univ...,41.949336
6,SELECT COUNT(*) FROM ROLES,How many roles are there?,CREATE TABLE ROLES (Id VARCHAR),2429,### Instruction:\n You are a powerful text-...,10\n### Solution:\n```sql\nSELECT COUNT(*) FRO...,16.610504
7,SELECT T1.text_of_notes FROM Assessment_Notes ...,Find the texts of assessment notes for teacher...,CREATE TABLE Assessment_Notes (text_of_notes V...,1840,### Instruction:\n You are a powerful text-...,SELECT * FROM Assessment_Notes WHERE teacher_i...,15.860643
8,"SELECT T1.first_name, T2.department_name FROM ...",Display the first name and department name for...,CREATE TABLE departments (department_name VARC...,2024,### Instruction:\n You are a powerful text-...,"SELECT e.first_name AS FirstName, d.department...",33.220817
9,SELECT T2.f1 FROM user_profiles AS T1 JOIN fol...,Find the id of users who are followed by Mary ...,"CREATE TABLE follows (f1 VARCHAR, f2 VARCHAR);...",170,### Instruction:\n You are a powerful text-...,SELECT uid FROM user_profiles WHERE EXISTS(SEL...,49.051716


(452, 7)

## Now let's calculate the Exact Match Accurcy For the Base LLaMA 2 Model

In [ ]:
import re
import pandas as pd
from difflib import SequenceMatcher
def normalize_text(text):
    """Remove punctuation, convert to lowercase, and strip extra spaces."""
    return re.sub(r'\W+', ' ', text).strip().lower()

def calculate_exact_match(df, threshold=0.90):
    def is_match(response, eval_res):
        normalized_response = normalize_text(response)
        normalized_eval = normalize_text(eval_res)
        similarity = SequenceMatcher(None, normalized_response, normalized_eval).ratio()
        return similarity >= threshold

    df['exact_match'] = df.apply(lambda row: is_match(row['response'], row['eval_res']), axis=1)
    accuracy = df['exact_match'].sum() / len(df)
    mismatches = df[~df['exact_match']]
    print(f"Exact Match Accuracy: {accuracy:.2%}")
    print(f"\nSample Mismatches ({len(mismatches)} cases):")

    return mismatches.sample(min(3, len(mismatches)))


In [28]:
mismatches = calculate_exact_match(test_df2)
display(mismatches)

Exact Match Accuracy: 39.60%

Sample Mismatches (273 cases):

,response,question,context,__index_level_0__,text,eval_res,eval_time,exact_match
20,"SELECT name, LOCATION, number_of_platforms FRO...","Show the name, location, and number of platfor...","CREATE TABLE station (name VARCHAR, LOCATION V...",3741,### Instruction:\n You are a powerful text-...,SELECT * FROM station;\n### Output:\n SELEC...,14.225626,False
69,"SELECT SUM(Population), MAX(GNP) FROM country ...",What is the total population and maximum GNP i...,"CREATE TABLE country (Population INTEGER, GNP ...",4356,### Instruction:\n You are a powerful text-...,SELECT Population FROM Country WHERE continent...,142.246319,False
342,"SELECT T2.Name, COUNT(*) FROM repair_assignmen...",Show names of technicians and the number of ma...,CREATE TABLE repair_assignment (technician_ID ...,1357,### Instruction:\n You are a powerful text-...,SELECT Name FROM technician WHERE technician_i...,81.781851,False
